# Transformando Variáveis Categóricas em Numéricas

Olá pessoal! 

Resolvi criar esse notebook para mostrar algumas das técnicas que conheço e utilizo para transformação de variáveis categóricas em numéricas.

O maior motivo para fazer essas transformações é que, na maioria dos modelos de ML, os dados precisam estar em formato numérico para funcionar. Por isso, é muito importante saber algumas formas de aplicar essas técnicas para conseguirmos trabalhar com qualquer dataset sem ter que codar muito e ser o mais produtivo possível.

Para esse "tutorial", vou usar um dos datasets mais famosos e explorados na comunidade de competições de IA do Kaggle: O Titanic (https://www.kaggle.com/c/titanic)

A tarefa é identificar quais os passageiros sobreviveriam ou não no acidente. Ou seja, uma tarefa de classificação.
Vamos começar importando as bibliotecas de i/o e os dados de treino e teste.

### Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

### Carregando os dados

In [2]:
treino = pd.read_csv('../datasets/train.csv')
teste = pd.read_csv('../datasets/test.csv')

treino_teste = [('treino', treino), ('teste', teste)]

Agora, vamos dar uma primeira olhada no nosso dataframe: 

In [3]:
# 5 primeiras amostras do df
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Nesse dataset, é fácil verificar quais são as variáveis que precisam de transformação, mas por via das dúvidas, é sempre bom usar o método `.info()` para verificar quais são as colunas do tipo `object`

In [4]:
# verificando colunas tipo object
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


De cara a gente já vê que existem alguns 'missing values'. Em outro momento, podemos nos aprofundar nas técnicas para tratar esse problema, mas hoje vamos focar na transformação numérica.

Para começar, vamos visualizar a coluna 'Sex' do nosso dataframe e fazer as primeiras transformações:

In [5]:
# visualisando coluna 'Sex'
treino[['Sex']]

,Sex
0,male
1,female
2,female
3,female
4,male
...,...
886,male
887,female
888,female
889,male


### Map + Dicionário

Umas das primeiras formas que aprendi a tratar as variáveis categórias, foi usando a função `map()` do pandas e o dicionário do python, da seguinte forma:

1. Criar um dicionário contendo o valor atual e o valor desejado;
2. Criar uma nova coluna utilizando a função `map(dicionario)`.

In [6]:
# criando o dicionário de dados
sexo_map = {'male': 0, 'female': 1}

# função map + dicio
treino['Sexo_map'] = treino['Sex'].map(sexo_map)

# mostrando coluna original e nova criada
treino[['Sex', 'Sexo_map']].head()

,Sex,Sexo_map
0,male,0
1,female,1
2,female,1
3,female,1
4,male,0


Ainda podemos usar uma função `lambda` para ter o mesmo resultado sem ter que escrever o dicionário

### Função Lambda

In [7]:
# criando lambda
treino['Sexo_lambda'] = treino['Sex'].map(lambda x: 0 if x == 'male' else 1)

# mostrando coluna original, Sex_map e Sex_lambda
treino[['Sex', 'Sexo_map', 'Sexo_lambda']].head()

,Sex,Sexo_map,Sexo_lambda
0,male,0,0
1,female,1,1
2,female,1,1
3,female,1,1
4,male,0,0


Essas são técnicas extremamente simples e fáceis de utilizar. 

O maior problema é que temos que, literalmente, criar um dicionário de dados e para cada chave única teremos um valor único do outro lado do " : ".

Imagine se tivéssemos que *mappear* uma coluna com 5 itens diferentes, ou então 10 itens distintos. Já viu a trabalheira né?

Você pode até estar pensando em criar um laço com todos os valores distintos e ir iterando e adicionando as *labels* neles. Porém, uma maneira mais rápida e eficiente é usar uma função muito legal da biblioteca scikit-learn do módulo de pré-processamento de dados: o LabelEncoder

## Usando Label Encoder - Scikit-Learn

doc: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

*sempre consulte a documentação, ajuda muito na hora de usar e na consistencia do seu código!*

In [8]:
# importando funcão
from sklearn.preprocessing import LabelEncoder

Agora que já temos a nossa função importada, é muito comum instanciarmos o objeto em uma variável:

In [9]:
# instanciando o objeto
label_encoder = LabelEncoder()

Dps de instanciado, vem a parte mais interessante dessa função: o método `fit(dados)`, onde dados, no nosso caso, é a coluna `treino['Sex']`

Esse método faz o objeto se ajustar à nossa coluna, automáticamente! Mas ainda não retorna nenhuma transformação, ele retorna apenas o objeto 'treinado':

In [10]:
# ajustando o objeto aos dados
label_encoder.fit(treino['Sex'])

LabelEncoder()

E agora? O que fazer com esse objeto 'treinado'?

Agora é só aplicar o método `transform(dados)` e agora sim, ele nos retornará uma array com os valores modificados:

In [11]:
# dados transformados
label_encoder.transform(treino['Sex'])

array([1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,

In [12]:
# criando nova coluna no dataframe
treino['Sexo_label_enc'] = label_encoder.transform(treino['Sex'])

# visualizando resultado
treino[['Sex', 'Sexo_map', 'Sexo_lambda', 'Sexo_label_enc']].head()

,Sex,Sexo_map,Sexo_lambda,Sexo_label_enc
0,male,0,0,1
1,female,1,1,0
2,female,1,1,0
3,female,1,1,0
4,male,0,0,1


#### Opa! Temos um erro!!

Calma, na verdade não é um erro. O método só está funcionando de forma inversa às nossas primeiras modificações. O que não modifica em nada no ponto de vista 'binário': ser ou não 'male' ou 'female'.

Se você achou o código muito grande, podemos dar um jeito nisso:

In [13]:
# usando método fit junto com o transform e criando nova coluna
treino['Sexo_label_enc2'] = label_encoder.fit_transform(treino['Sex']) #1 única linha de código

# visualizando resultado
treino[['Sex', 'Sexo_map', 'Sexo_lambda', 'Sexo_label_enc', 'Sexo_label_enc2']].head()

,Sex,Sexo_map,Sexo_lambda,Sexo_label_enc,Sexo_label_enc2
0,male,0,0,1,1
1,female,1,1,0,0
2,female,1,1,0,0
3,female,1,1,0,0
4,male,0,0,1,1


Pra ficar mais legal, vamos agora testar esse mesmo método com uma outra coluna do nosso dataset. A coluna 'Embarked' possui o porto onde cada passageiro embarcou no Titanic. São 3 portos possíveis: C, S e Q. Vamos usar o nosso `label_encoder` para transformação.

In [14]:
#criando nova coluna, e transformando os valores
treino['Embarked_Label_Enc'] = label_encoder.fit_transform(treino['Embarked'])

#visualizando o resultado
treino[['Embarked', 'Embarked_Label_Enc']]

,Embarked,Embarked_Label_Enc
0,S,2
1,C,0
2,S,2
3,S,2
4,S,2
...,...,...
886,S,2
887,S,2
888,S,2
889,C,0


Viu como foi simples?

Mas existe um problema com esse tipo de transformação. Como os nossos modelos de ML são matemáticos, é possível que ele entenda essa transformação como **categorias ordinais** e isso pode fazer com o que ele atribua **pesos diferentes de acordo com a ordem**. 

Um exemplo de categoria ordinal pode ser: "pouco", "normal" e "muito". Nesse caso, faria sentido a transformação tbm obedecer uma "ordem de grandeza".

Mas no caso dos portos, essa diferença não existe... E agora??

### One Hot Enconding e Dummy Variables

Para resolver esse problema, podemos recorrer a uma outra técnica de transformação que se chama "one hot variables" ou "dummy variables". 

Vamos usar novamente o sk-learn para resolver esse problema.

In [15]:
# importando biblioteca
from sklearn.preprocessing import OneHotEncoder

# instanciando o objeto
ohe = OneHotEncoder(sparse=False) 

Como você viu, precisamos passar o argumento `False` para o sparse para que o objeto nos retorne um array.

In [16]:
ohe.fit_transform(treino[['Embarked']])

array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

Como vamos transformar 1 coluna em N colunas, o ideal é criar um novo dataFrame com essas novas colunas e juntar ao dataframe antigo. Assim:

In [17]:
# criando variavel com os valores transformados
valores_ohe = ohe.fit_transform(treino[['Embarked']])

# criando novo dataframe com os valores transformados
ohe_embarked = pd.DataFrame(data=valores_ohe)

# visualisando o resultado
ohe_embarked

,0,1,2,3
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
886,0.0,0.0,1.0,0.0
887,0.0,0.0,1.0,0.0
888,0.0,0.0,1.0,0.0
889,1.0,0.0,0.0,0.0


In [18]:
# juntando os dois dataframes
treino[['Embarked']].join(ohe_embarked)

,Embarked,0,1,2,3
0,S,0.0,0.0,1.0,0.0
1,C,1.0,0.0,0.0,0.0
2,S,0.0,0.0,1.0,0.0
3,S,0.0,0.0,1.0,0.0
4,S,0.0,0.0,1.0,0.0
...,...,...,...,...,...
886,S,0.0,0.0,1.0,0.0
887,S,0.0,0.0,1.0,0.0
888,S,0.0,0.0,1.0,0.0
889,C,1.0,0.0,0.0,0.0


Como descobrir qual coluna pertençe a qual classe?

O objeto treinado possui o atributo `features_names()` que possui a os nomes das categorias transformadas. Entao é só passar esse argumento na criação do novo DataFrame e as colunas serão renomeadas:

In [19]:
ohe.get_feature_names()

array(['x0_C', 'x0_Q', 'x0_S', 'x0_nan'], dtype=object)

In [20]:
ohe_embarked = pd.DataFrame(data=valores_ohe, columns=list(ohe.get_feature_names()))
ohe_embarked.head()

,x0_C,x0_Q,x0_S,x0_nan
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


Parece complicado né? Mas com a prática você se acostuma e faz bem rápido. 

Pra finalizar, o OneHotEncoder possui um parâmetro bem interessante que é o `drop=`. Nele, podemos determinar quando 'dropar' (excluir) uma coluna. 

In [21]:
# dropando a 1ª coluna
ohe2 = OneHotEncoder(drop='first', sparse=False)

In [22]:
# criando variavel com os valores transformados
valores_ohe2 = ohe2.fit_transform(treino[['Embarked']]).astype(int) # astype(int) para transformar os valores em inteiros

# criando novo dataframe com os valores transformados
ohe_embarked2 = pd.DataFrame(data=valores_ohe2, columns=list(ohe2.get_feature_names()))

# visualisando o resultado
treino[['Embarked']].join(ohe_embarked2)

,Embarked,x0_Q,x0_S,x0_nan
0,S,0,1,0
1,C,0,0,0
2,S,0,1,0
3,S,0,1,0
4,S,0,1,0
...,...,...,...,...
886,S,0,1,0
887,S,0,1,0
888,S,0,1,0
889,C,0,0,0


Se a categoria não é **Q**, não é **S** e não é **Nulo**, ele só pode ser o **C**. Por isso, podemos excluir a coluna pois ela traz uma informação redundante. Existem outras formas de drop que vc pode consultar na documentação.

# pd.get_dummies

Para finalizar, e agora é de verdade (kkk), o pandas tem um método de dataframe que faz exatamente a mesma coisa e que por muitas vezes eu acho mais fácil de usar:

In [23]:
dummy_embarked = pd.get_dummies(treino[['Embarked']])
dummy_embarked

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
886,0,0,1
887,0,0,1
888,0,0,1
889,1,0,0


In [24]:
treino[['Embarked']].join(dummy_embarked)

,Embarked,Embarked_C,Embarked_Q,Embarked_S
0,S,0,0,1
1,C,1,0,0
2,S,0,0,1
3,S,0,0,1
4,S,0,0,1
...,...,...,...,...
886,S,0,0,1
887,S,0,0,1
888,S,0,0,1
889,C,1,0,0


In [26]:
dummy_drop = pd.get_dummies(treino['Embarked'], drop_first=True)
treino[['Embarked']].join(dummy_drop)

,Embarked,Q,S
0,S,0,1
1,C,0,0
2,S,0,1
3,S,0,1
4,S,0,1
...,...,...,...
886,S,0,1
887,S,0,1
888,S,0,1
889,C,0,0
